# Ensemble CNNs + CatBoost + LGBM models

## Imports

In [26]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import models
from catboost import CatBoostClassifier
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
import torchvision.transforms as transforms


import cv2
import numpy as np
import h5py
from tqdm import tqdm
import io
import random
import pandas as pd
from PIL import Image


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [10]:
SEED = 111
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

ROOT_DIR = "../data"
TRAIN_CSV = f"{ROOT_DIR}/train-metadata.csv"
TRAIN_HDF = f"{ROOT_DIR}/train-image.hdf5"
TEST_CSV = f'{ROOT_DIR}/test-metadata.csv'
TEST_HDF = f'{ROOT_DIR}/test-image.hdf5'
SAMPLE = f'{ROOT_DIR}/sample_submission.csv'

In [11]:
CONFIG = {
    "seed": 42,
    "epochs": 20,
    "img_size": 224,
    "train_batch_size": 150,
    "valid_batch_size": 200,
    "learning_rate": 1e-5,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-6,
    "T_max": 500,
    "weight_decay": 1e-6,
    "fold" : 0,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": device,

    }

## Init CNN Architectures

In [12]:
"""
RESNET152
"""

class CustomResNet152(nn.Module):
    def __init__(self):
        super(CustomResNet152, self).__init__()
        # Load a pre-trained ResNet-152 model
        self.base_model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
        
        # remove last fully connected layer for our purposes
        self.features = nn.Sequential(*list(self.base_model.children())[:-2])

        # Classifier that includes flattening the feature map and linear layer for class prediction
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 1),  # output single value for prob calculation
            nn.Sigmoid()  # sigmoid activation for probability
        )
    
    def forward(self, x):
        # extract features
        x = self.features(x)
        # classify features
        output = self.classifier(x)
        return output

model1 = CustomResNet152()

In [13]:
"""
RESNET50
"""

class CustomResNet50(nn.Module):
    def __init__(self):
        super(CustomResNet50, self).__init__()
        # Load a pre-trained ResNet-152 model
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # remove last fully connected layer for our purposes
        self.features = nn.Sequential(*list(self.base_model.children())[:-2])

        # Classifier that includes flattening the feature map and linear layer for class prediction
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 1),  # output single value for prob calculation
            nn.Sigmoid()  # sigmoid activation for probability
        )
    
    def forward(self, x):
        # extract features
        x = self.features(x)
        # classify features
        output = self.classifier(x)
        return output

model2 = CustomResNet50()

In [14]:
"""
MobileNetV2
"""

class CustomMobileNetV2(nn.Module):
    def __init__(self):
        super(CustomMobileNetV2, self).__init__()
        self.base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        
        self.features = self.base_model.features
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(1280, 1),
            nn.Sigmoid()  
        )
    
    def forward(self, x):
        x = self.features(x)
        output = self.classifier(x)
        return output

model3 = CustomMobileNetV2()

In [15]:
"""
MNASNet-1.0
"""

class CustomMNASNet(nn.Module):
    def __init__(self):
        super(CustomMNASNet, self).__init__()
        self.base_model = models.mnasnet1_0(weights=models.MNASNet1_0_Weights.IMAGENET1K_V1)
        
        self.features = self.base_model.layers
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(1280, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        output = self.classifier(x)
        return output
    
model4 = CustomMNASNet()

In [16]:
"""
EfficientNetB4
"""

class CustomEfficientNetB4(nn.Module):
    def __init__(self):
        super(CustomEfficientNetB4, self).__init__()
        self.base_model = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.IMAGENET1K_V1)
        
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(p=0.4), 
            nn.Linear(self.base_model.classifier[1].in_features, 1),
            nn.Sigmoid() 
        )
    
    def forward(self, x):
        output = self.base_model(x)
        return output
    
model5 = CustomEfficientNetB4()

In [17]:
"""
DenseNet121
"""

class CustomDenseNet121(nn.Module):
    def __init__(self):
        super(CustomDenseNet121, self).__init__()
        self.base_model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        
        self.base_model.classifier = nn.Sequential(
            nn.Linear(1024, 1), 
            nn.Sigmoid()
        )
    
    def forward(self, x):
        output = self.base_model(x)
        return output

model6 = CustomDenseNet121()

## Load CNNs in

In [18]:
model1.load_state_dict(torch.load('./output/res152_ISIC_best.pth', map_location=torch.device('cuda')))
model2.load_state_dict(torch.load('./output/res50_ISIC_best.pth', map_location=torch.device('cuda')))
model3.load_state_dict(torch.load('./output/mobileV2_ISIC_best.pth', map_location=torch.device('cuda')))
model4.load_state_dict(torch.load('./output/mnas1_0_ISIC_best.pth', map_location=torch.device('cuda')))
model5.load_state_dict(torch.load('./output/effB4_ISIC_best.pth', map_location=torch.device('cuda')))
model6.load_state_dict(torch.load('./output/Dense121_ISIC_best.pth', map_location=torch.device('cuda')))

<All keys matched successfully>

## Load CatBoost and LGBM in

In [19]:
"""
Catboost
"""

model7 = CatBoostClassifier()
model7.load_model("./output/catboost_model.cbm")

In [20]:
"""
LGBM
"""

model8 = lgb.Booster(model_file="./output/lightgbm_model.txt")

## Reinput data

In [27]:
"""
import just the testing dataset
"""

def read_images_from_hdf5(file_path):
    images = {}
    try:
        with h5py.File(file_path, 'r') as file:
            for key in tqdm(file.keys(), desc="Reading Files"):
                try:
                    image_data = file[key][()]
                    image = Image.open(io.BytesIO(image_data))
                    images[key] = image
                except Exception as e:
                    print(f"Error! from {key}: {e}")
    except Exception as e:
        print(f"Error occured while reading files : {e}")
    
    return images

test_images = read_images_from_hdf5(TEST_HDF)
test_metadata = pd.read_csv(TEST_CSV)

Reading Files: 100%|██████████| 3/3 [00:00<00:00, 768.23it/s]


In [36]:
"""
import test image dataloaders for CNN evaluation
"""

def remove_hair(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    blackhat = cv2.morphologyEx(gray,cv2.MORPH_BLACKHAT,kernel)

    _, thresh = cv2.threshold(blackhat, 10 ,255, cv2.THRESH_BINARY)
    inpainted_image = cv2.inpaint(image, thresh, 1, cv2.INPAINT_TELEA)
    return inpainted_image


class ISIC_2024(Dataset):
    def __init__(self,pil_images,metadata,transform=None,test=False):
        self.pil_images = pil_images
        self.metadata = metadata
        self.transform = transform
        self.test= test
        
    def __len__(self):
        return len(self.metadata)
    
    # This function from https://www.kaggle.com/competitions/isic-2024-challenge/discussion/519735
    def remove_hair(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
        blackhat = cv2.morphologyEx(gray,cv2.MORPH_BLACKHAT,kernel)

        _, thresh = cv2.threshold(blackhat, 10 ,255, cv2.THRESH_BINARY)
        inpainted_image = cv2.inpaint(image, thresh, 1, cv2.INPAINT_TELEA)
        return inpainted_image
    
    def __getitem__(self,idx):
        isic_id = self.metadata.iloc[idx,0]
        cleaned_image = remove_hair(np.array(self.pil_images[isic_id]))
        image = Image.fromarray(cleaned_image)
        if self.transform:
            image = self.transform(image)
        if self.test:
            return image, isic_id
        label = self.metadata.iloc[idx,-1]
        return image,label,isic_id

test_transforms = transforms.Compose([
     transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # commonly used mean and std calculated from ImageNet
])

In [37]:
test_dataset = ISIC_2024(test_images, test_metadata, transform = test_transforms, test = True)
test_loader = DataLoader(test_dataset, batch_size=32, pin_memory=True)

In [40]:
for batch in test_loader:
    # Check the length of the batch
    
    # Adjust based on what your dataset returns
    if len(batch) == 2:
        images, ids = batch
    elif len(batch) == 3:
        images, ids, labels = batch  # Assuming the third element is the label
    # Add more conditions if necessary
    
    # Use the images and ids as needed
    print(ids)
    break  # Just to check one iteration

['ISIC_0015657', 'ISIC_0015729', 'ISIC_0015740']


In [42]:
for images, ids in test_loader:
    print(ids)

['ISIC_0015657', 'ISIC_0015729', 'ISIC_0015740']
